In [41]:
import torch
from pointnet.model import PointNet

state = torch.load("logs/baseline l=5/model_weights.pt")
l = int(state["coef_mean"].shape[0]**0.5)-1
batch_norm = "bn1.weight" in state
model = PointNet(l,batch_norm=batch_norm)
model.load_state_dict(state)
model.eval()


traced_model = torch.jit.trace_module(model,{
    "forward":[torch.rand((1,3)),torch.rand((1,3)),torch.zeros(1,dtype=torch.long)]
})

torch.jit.save(traced_model,"traced_model_cpu.pt")

traced_model = torch.jit.trace_module(model.cuda(),{
    "forward":[torch.rand((1,3)).cuda(),torch.rand((1,3)).cuda(),torch.zeros(1,dtype=torch.long).cuda()]
})

torch.jit.save(traced_model,"traced_model_gpu.pt")

In [38]:
traced_model.cuda()(torch.rand((1,3)).cuda(),torch.rand((1,3)).cuda(),torch.zeros(1,dtype=torch.long).cuda())

tensor([[[ 5.4547e+01,  5.5027e+01,  5.7327e+01,  1.6335e+01],
         [ 4.0035e+00,  3.7514e+00,  3.4781e+00, -2.1461e-04],
         [-1.0867e+00, -1.0122e+00, -9.1774e-01, -1.8028e-04],
         [-4.0898e+00, -3.8562e+00, -3.5988e+00, -3.1750e-04],
         [-2.5995e+00, -2.4287e+00, -2.2089e+00,  5.5148e-01],
         [-1.1517e-01, -2.2008e-01, -3.6679e-01,  6.1407e-01],
         [ 3.1137e+00,  3.0624e+00,  2.9588e+00,  1.1256e+00],
         [-2.6889e+00, -2.5908e+00, -2.4574e+00,  4.8739e-01],
         [-1.5829e+00, -1.4517e+00, -1.2755e+00,  2.0893e+00],
         [ 2.5119e-01,  2.5355e-01,  2.4300e-01,  8.1895e-04],
         [ 1.5103e-01,  1.2334e-01,  1.2166e-01,  1.2679e-04],
         [-8.2110e-02, -5.4228e-02, -4.2781e-02,  1.4317e-04],
         [-4.5630e-02, -5.3959e-02, -5.3810e-02, -2.5628e-04],
         [-2.4521e-01, -2.1533e-01, -1.9092e-01, -9.1105e-05],
         [ 2.7862e-01,  2.6950e-01,  2.6084e-01,  1.7393e-04],
         [ 1.4899e-01,  1.3932e-01,  1.7041e-01, -1.017

In [ ]:
for b in traced_model.buffers():
    print(b.device)

In [ ]:
from pointnet.dataset import OctantDataset

ds = OctantDataset("datasets/neuschwanstein/octants_1024max_sh",preload=True)

ds

In [ ]:
from pointnet.model import PointNet
import torch

model = PointNet(10,4,batch_norm=True,use_dropout=True)
model.eval()
model.load_state_dict(torch.load("logs/100k top_variance, with_alpha, camera_loss only, small dropout/model.pt"))

traced_model = torch.jit.trace_module(model,{
    "forward":[torch.rand((1,3)),torch.rand((1,4)),torch.zeros(1,dtype=torch.long)]
})

torch.jit.save(traced_model,"traced_model.pt")

In [ ]:
model =torch.jit.load("logs/100k top_variance l=5/traced_model.pt").cuda()
model.eval()

In [ ]:
model(torch.rand((1,3)).cuda(),torch.rand((1,4)).cuda(),torch.zeros(1,dtype=torch.long).cuda())

In [ ]:
from tqdm import tqdm
from pointnet.pointclouds import collate_batched

with open("datasets/neuschwanstein/octants_1024max_sh/coefs.raw","wb") as f:
    batches = torch.arange(len(ds)).split(256)
    for batch in tqdm(batches):
        pc = collate_batched([ds[i][0] for i in batch])
        coefs = model(pc.points_packed().cuda(),pc.features_packed().cuda(),pc.packed_to_cloud_idx().cuda())
        octant_id = int(pc.file_names[0].split("/")[-1].split(".")[0].split("_")[1])
        f.write(octant_id.to_bytes(8,"little"))
        f.write(coefs.detach().flatten().cpu().numpy().tobytes())

In [43]:
x = torch.ones(125,3)
torch.cat([x,x],dim=0).shape

torch.Size([250, 3])